# Install detectron2

In [0]:
# Optional, if you need your Google Drive, uncomment it
#from google.colab import drive
#drive.mount('/content/drive')

In [13]:
!pip install -U torch torchvision cython
!pip install -U 'git+https://github.com/facebookresearch/fvcore.git' 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

Requirement already up-to-date: torch in /usr/local/lib/python3.6/dist-packages (1.3.1)
Requirement already up-to-date: torchvision in /usr/local/lib/python3.6/dist-packages (0.4.2)
Requirement already up-to-date: cython in /usr/local/lib/python3.6/dist-packages (0.29.14)
  Cloning https://github.com/facebookresearch/fvcore.git to /tmp/pip-req-build-d7mqzd42
  Running command git clone -q https://github.com/facebookresearch/fvcore.git /tmp/pip-req-build-d7mqzd42
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-qpcq34_7
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-qpcq34_7
  Created wheel for fvcore: filename=fvcore-0.1-cp36-none-any.whl size=34040 sha256=44ed3e1296b492652e184d7af707f24a86bc90ab716bbc6a3a163f61617617fe
  Stored in directory: /tmp/pip-ephem-wheel-cache-u_4n9mqk/wheels/48/53/79/3c6485543a4455a0006f5db590ab9957622b6227011941de06
  Created wheel for pycocotools: filename=pycocotools-2.0-cp36-cp36m-lin

'1.3.1'

In [14]:
!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!pip install -e detectron2_repo

fatal: destination path 'detectron2_repo' already exists and is not an empty directory.
Obtaining file:///content/detectron2_repo
  Found existing installation: detectron2 0.1
    Can't uninstall 'detectron2'. No files were found to uninstall.
  Running setup.py develop for detectron2


In [1]:
import torch, torchvision
torch.__version__

'1.3.1'

In [0]:
# You may need to restart your runtime prior to this, to let your installation take effect
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer

import os
import json
from itertools import groupby
import numpy as np
import cv2
import random

from pycocotools.coco import COCO
from pycocotools import mask as maskutil

In [0]:
# instead of "/content/drive/My Drive/VRDL_HW4/pascal_train.json" and 
# "/content/drive/My Drive/VRDL_HW4/train_images" put your file and folder path
register_coco_instances("VOC", {}, "/content/drive/My Drive/VRDL_HW4/pascal_train.json", "/content/drive/My Drive/VRDL_HW4/train_images")
VOC_metadata = MetadataCatalog.get("VOC")

# Train on a custom dataset

In [4]:
cfg = get_cfg()
cfg.merge_from_file("./detectron2_repo/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.DATASETS.TRAIN = ("VOC",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.WEIGHT_DECAY_BIAS = 0.01
cfg.SOLVER.CHECKPOINT_PERIOD = 3000
cfg.SOLVER.MAX_ITER = 30000
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 20

WARNING [12/19 15:36:03 d2.config.compat]: Config './detectron2_repo/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml' has no VERSION. Assuming it to be compatible with latest v2.


In [0]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

In [0]:
def binary_mask_to_rle(binary_mask):
    rle = {'counts': [], 'size': list(binary_mask.shape)}
    counts = rle.get('counts')
    for i, (value, elements) in enumerate(groupby(binary_mask.ravel(order='F'))):
        if i == 0 and value == 1:
            counts.append(0)
        counts.append(len(list(elements)))
    compressed_rle = maskutil.frPyObjects(rle, rle.get('size')[0], rle.get('size')[1])
    compressed_rle['counts'] = str(compressed_rle['counts'], encoding='utf-8')
    return compressed_rle

In [0]:
# instead of "/content/drive/My Drive/VRDL_HW4/pascal_train.json" put your weigths file path
cfg.MODEL.WEIGHTS = "/content/drive/My Drive/VRDL_HW4/model_final_resnet50_w_decay.pth"
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set the testing threshold for this model
cfg.DATASETS.TEST = ("VOC")
CLASSES = ["aeroplane", "bicycle", "bird", "boat", "bottle",
                "bus", "car", "cat", "chair", "cow",
                "diningtable", "dog", "horse", "motorbike", "person",
                "pottedplant", "sheep", "sofa", "train", "tvmonitor"]
VOC_metadata.set(thing_classes=CLASSES)
predictor = DefaultPredictor(cfg)

In [8]:
JSON_OUTPUT = "0780828.json" # the name of submission json file

# instead of "/content/drive/My Drive/VRDL_HW4/test.json" and
# "/content/drive/My Drive/VRDL_HW4/test_images/" put your file and folder path
coco_test = COCO("/content/drive/My Drive/VRDL_HW4/test.json")
coco_result = []
for imgid in coco_test.imgs:
    image = cv2.imread("/content/drive/My Drive/VRDL_HW4/test_images/" + coco_test.loadImgs(ids=imgid)[0]['file_name'])[:, :, ::-1]
    outputs = predictor(image)
    scores = outputs["instances"].scores
    for i in range(len(scores)):
        pred = {}
        pred['image_id'] = imgid
        pred['category_id'] = int(outputs["instances"].pred_classes[i] + 1)
        binary_mask = outputs["instances"].pred_masks[i].to("cpu").numpy()
        pred['segmentation'] = binary_mask_to_rle(binary_mask)
        pred['score'] = float(outputs["instances"].scores[i])
        coco_result.append(pred)

with open(JSON_OUTPUT, "w") as f:
    json.dump(coco_result, f)
print("Done")

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Done
